In [6]:
# Get Connector JAR name
import glob
import os

files = glob.glob("/spark-connector/connector/target/scala-2.12/spark-vertica-connector-assembly-*")
os.environ["CONNECTOR_JAR"] = files[0]
print(os.environ["CONNECTOR_JAR"])

/spark-connector/connector/target/scala-2.12/spark-vertica-connector-assembly-3.3.4.jar


In [7]:
# Create the Spark session and context
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .config("spark.master", "spark://spark:7077")
    .config("spark.driver.memory", "2G")
    .config("spark.executor.memory", "1G")
    .config("spark.jars", os.environ["CONNECTOR_JAR"])
    .getOrCreate())
sc = spark.sparkContext

In [8]:
# Display the context information
print(sc.version)
print(sc.master)
display(sc.getConf().getAll())

3.3.0
spark://spark:7077


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.driver.memory', '2G'),
 ('spark.repl.local.jars',
  'file:///spark-connector/connector/target/scala-2.12/spark-vertica-connector-assembly-3.3.4.jar'),
 ('spark.app.initial.jar.urls',
  'spark://1737e2b500fa:35359/jars/spark-

In [9]:
# Perform a write with complex arrays, then read using the Spark Connector
from pyspark.sql.types import StringType, ArrayType, StructType, StructField, IntegerType, Row

schema = StructType([StructField("native_array", ArrayType(IntegerType())), 
                     StructField("nested_array", ArrayType(ArrayType(IntegerType()))),
                     StructField("internal_map", ArrayType(StructType([
                              StructField("key", StringType()),
                              StructField("value", IntegerType()),
                            ])))])
data = [Row([1234567], [[1,2], [3, 4]], [["key1", 5], ["key2", 6]])]
df = spark.createDataFrame(spark.sparkContext.parallelize(data), schema).coalesce(1)

df.write.mode("overwrite").format("com.vertica.spark.datasource.VerticaSource").options(
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="Complex_Array_Examples",
    staging_fs_url="webhdfs://hdfs:50070/complextypes").save()

df = spark.read.load(format="com.vertica.spark.datasource.VerticaSource",
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="Complex_Array_Examples",
    staging_fs_url="webhdfs://hdfs:50070/complextypes")
df.rdd.collect()
df.show()

+------------+----------------+--------------------+
|native_array|    nested_array|        internal_map|
+------------+----------------+--------------------+
|   [1234567]|[[1, 2], [3, 4]]|[{key1, 5}, {key2...|
+------------+----------------+--------------------+

